In [1]:
import json
#load the json file for the source file
with open('xpo_v5.3.json','r') as f:
    parsed_j=json.load(f)

In [2]:
import re

In [3]:
def get_arg_name(args):
    '''
    remove the middle part, like 'A0_ppt_buried_entity'  to "A0_buried_entity" for more readable
    '''
    if args.startswith('AM'):  # AM is a kind of exception, so just keep the original format 
        arg=args
    else:
        strings=args.split('_')
        arg=strings[0]+'_'+'_'.join(strings[2:])# remove the second part after spliting by underscore
    return arg

In [9]:
import pandas as pd
# pattern files , which offer the preposition for each arguments
prep=pd.read_excel("dwd templates for json (version 1) (1).xlsx",header=None,engine='openpyxl')

In [5]:
re.split('<.*?>','with <instrument or means>')

['with ', '']

In [6]:
def remove_arg(pd): # original format  with A2 , format to with
    if '<' in pd[1]:
        pd[1]=re.split('<.*?>',pd[1])[0]
    else:
        pd[1]=pd[1].split()[0]
    
    return pd

In [57]:
for k,v in dict_prep.items():
    print(k,v)

A1_com with
A1_loc_path preposition
A1_loc_direction preposition
A2_mnr with
AM_loc at
A2_gol_listener to
A2_gol_hearer to
A2_gol_recipient to
A2_gol_destination to
A2_gol_appealed_to to
A2_gol_target at
A2_gol_benef for
A2_gol_entity_provided_for for
A2_gol_co-agent with
A2_gol_victim against
A2_ppt PREPOSITION
A2_vsp PREPOSITION
A2_com with
A2_dir from
A2_tmp until
A2_ext (by)
A2_prd_end_state into
A2_prd_product into
A2_prd_secondary_attribute as
A2_prd_attribute as
A2_prd_for_what for
A2_prd_blessed_with with
A2_prd_mislead_into into
A2_prd_into_pieces into
A2_prd_value at
A2_prd (PREPOSITION)
A2_cau_cause_or_disease with
A2_cau_protected_from from
A2_cau_secondary_attribute as
A2_cau_fighting_about about
A2_cau_topic_or_cause_of_fighting about
A2_cau for
A2_adj for
A3_prd_attribute as
A3_prd_secondary_attribute as
A3_prd_end_state into
A3_prd_product into
A3_prd_attributive on
A3_prd PREPOSITION
A3_mnr with
A3_gol_electorate_for_whom_arg0_is_a_proxy_voter A3
A3_gol_listener to
A3_

In [22]:
prep.head() # look the first sevreral rows

,0,1
0,A1_com,with
1,A1_loc_path,preposition
2,A1_loc_direction,preposition
3,A2_mnr,with
4,AM_loc,at


In [7]:
new_prep=prep.apply(remove_arg,axis=1)# apply the function to each row

In [8]:
dict_prep=dict(zip(new_prep.iloc[:,0],new_prep.iloc[:,1]))  # convert the argument and the corresponding preposition to a dict

In [52]:
import requests
from bs4 import BeautifulSoup
count=0 # id for the rows
error=[]
events=parsed_j['events'] # from the json file get the events
df=pd.DataFrame(columns=('qnode_id','event_type','template','no_propbank','no_verb','has_A5'))  # create a bew dataframe for creating the csv
for qnode,event in events.items():# iterate the events
    marked=''
    template=''
    no_prob=False
    no_verb=3
    # print(qnode,event['name'],':',end='')

    event_type=event['name']
    arguments=event['arguments']
    left_args=[]
    verbs= event['name']  # determine the verb for each events
    if 'pb_roleset' in event.keys():
        # print(event['name'])
        url='https://verbs.colorado.edu/propbank-development/'+event['pb_roleset'].split('.')[0]+'.html'
        # print(url)
        # headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.119 Safari/537.36'}
        html=requests.get(url)
        if str(html)=='<Response [404]>':
            url='https://verbs.colorado.edu/propbank-development/'+'alias-'+event['pb_roleset'].split('.')[0]+'.html'
            try:
                 html=requests.get(url)
            except:
                error.append(qnode)
        soup=BeautifulSoup(html.text)
        try:
            all_alias=soup.body.find_all("div", {"class": "wrapper"})[0].find_all('div',{'class':'content'})[0].find_all('div',{'class':'sticky-content'})[0].h4.find_next_siblings()[0].select('span')
        except:
            error.append(qnode)
        no_verb=2
        for alias in all_alias:
            imatch_pos=re.search('\((.*)\)',alias.text)
            if imatch_pos.group(1)=='v.':
                verbs=alias.text.strip('(v.)')
                no_verb=1  
                break
    else:
        no_prob=True
            # if imatch_pos:
            #     print(imatch_pos.group[1])
    if len(arguments)==1:  # for events just have one arguments
        template= f"<{get_arg_name(arguments[0]['name'])}> {verbs}" 
    else:# for others, just look up the prepositions and concatenate with the argument name
        template=''
        if arguments[1]['name'] in dict_prep :
            arg1=f"{dict_prep[arguments[1]['name']]} <{get_arg_name(arguments[1]['name'])}>"
        elif arguments[1]['short_name'] in dict_prep:
            arg1=f"{dict_prep[arguments[1]['short_name']]} <{get_arg_name(arguments[1]['name'])}>"
        else:
            arg1=f"<{get_arg_name(arguments[1]['name'])}>"
        template+=f"<{get_arg_name(arguments[0]['name'])}> {verbs} {arg1}"
        for i, argument in enumerate(arguments[2:]):
            if argument['name'] in dict_prep:
                template+= f" {dict_prep[argument['name']]} <{get_arg_name(argument['name'])}>"
            elif argument['short_name'] in dict_prep:
                  template+=f" {dict_prep[argument['short_name']]} <{get_arg_name(argument['name'])}>"
            else:
                template +=f" <{get_arg_name(argument['name'])}>"
            if argument['short_name'].startswith('A5'):  # for some reasons, to mark the events has more than 5 arguments 
                marked=1
  
    df.loc[count]={'qnode_id':qnode,'event_type':event_type,'template':template,"no_propbank":1 if no_prob is True else '','no_verb':1 if no_verb ==2 else '','has_A5':1 if marked==1 else ''} # insert the row to the dataframe

    # print("*"*22)
    count+=1
   

In [53]:
for err in error:
    print(err)

DWD_Q873506


In [54]:
df.head()# peek the dataframe

,qnode_id,event_type,template,no_propbank,no_verb,has_A5
0,DWD_Q7216866,post,<A0_transmitter> post <A1_message> to <A2_audi...,1,,
1,DWD_Q56119332,tweet,<A0_transmitter> tweet <A1_message> to <A2_aud...,1,,
2,DWD_Q34394,belief,<A0_believer> believe <A1_believed> PREPOSITI...,,,
3,DWD_Q190084,hoax,<A0_hoaxer> hoax <A1_target> (PREPOSITION) <A...,,,
4,DWD_Q34302,doubt,<A0_disbeliever> doubt <A1_disbelief>,,,


In [55]:
df.to_csv('template_info.csv')  # save as csv

In [5]:
from collections import defaultdict
count=0
passive=0
passive_name=[]
events=parsed_j['events'] # from the json file get the events
AM_dict=defaultdict(int)
num_arguments=defaultdict(int)
for qnode,event in events.items():# iterate the events

    event_type=event['name']
    arguments=event['arguments']
    if arguments[0]['name'].startswith('A1'):
        passive+=1
        passive_name.append(event_type)
    count+=1
    num_arguments[len([args for args in arguments if not args['short_name'].startswith('AM')])]+=1
    for i,j in enumerate(arguments):
        if j['short_name'].startswith('AM'):
            AM_dict[j['name']]+=1



In [13]:
count

4706

In [12]:
print(f"the passive events is {(passive/count)*100:.2f} %")
print(f"the passive events are :")
for i in passive_name:
    print(i, end=', ')

the passive events is 7.20 %
the passive events are :
scarcity, disease_outbreak, conflagration, merger, mutation, natural_causes, death, disease, symptom, health_problem, abdominal_distention, abdominal_exercise, abdominal_symptom, acarodermatitis, accidental_death, acropustulosis, actinomycosis, adverse_event, affair, aftershock, air_current, air_current, airflow, airprox, alcoholism, animal_disease, animalisation, ankyloglossia, annual_event, anticyclone, apparition, approach, arthropod_dermatitis, associated_diseases, asthenia, astronomical_phenomenon, astronomical_transit, asymmetry, atmospheric_phenomenon, atrophy, attenuation, avalanche, backpacking, bacterial_myositis, banking_crisis, beginning, binge-eating_disorder, biogeochemical_process, blizzard, blood_flow, bond, budding, bushfire, cardiac_fibrillation, carnaval_parade, cavalcade, cell_death, chance, charity_event, chills, circulatory_collapse, clear-air_turbulence, cloudburst, cognitive_disorder, coincidence, cold_wave, 

In [14]:
for k,v in sorted(AM_dict.items(),key=lambda x:x[1],reverse=True):
    print(k.lstrip('AM_'),v)

loc 4677
mnr__manner 39
dir_direction 10
cau__cause 9
loc_medium 9
loc_Medium 6
tmp_point_at_which_something_ends 5
gol__goal 4
dir__directional 3
mnr_instrument 2
loc_direction_or_location 2
loc_location_of_meeting 2
tmp_time_of_meeting 2
com__comitative 1
loc_domain_of_competition 1
loc_direction 1
dir_path_or_medium 1
tmp_period_of_time_waited_through 1


In [84]:

for k,v in sorted(num_arguments.items(),key=lambda x:x[1],reverse=True):
    print(k,v)

3 2061
2 1550
4 764
5 184
1 122
6 25


In [15]:
import plotly.graph_objects as go
import plotly.express as px

In [16]:
def plot_graph(tdict, title,x_name,y_name):
    # Create a figure we will add our chart to
    fig = go.Figure()

    # Add a bar chart to the figur
    fig.add_trace(go.Bar(x = [i for i in tdict.keys()],                                # x-axis data
                         y = [i for i in tdict.values()],                                # y-axis data
                         text=[str(i) for i in tdict.values()],                               # Textual data displayed above each bar
                         textposition='outside',                            # Location of 'text'
                         orientation='v',                                   # We want veritcle bars, and 'h' would give us horizontal bars
                        )
                 )

    fig.update_layout(title=f'<b>{title} Analysis</b>',                     # Title of figure/chart
                      width = 1400,                                         # Width of figure
                      height = 800,                                         # Height of figure
                      xaxis_title=f"<b>{x_name}</b>",                          # x-axis label
                      yaxis_title=f"<b>{y_name}</b>",    # y-axis label
                      font=dict(family="Arial",                             # dict. specifing font details
                                size=16,
                            )
                     )

    fig.show()          

In [18]:
plot_graph(num_arguments,'distribution of the umber of arguments for each events','number of arguments','number of events')

In [17]:
plot_graph(AM_dict,'distribution of the umber of non-core arguments','type-of non-core arguments','number of events')